In [1]:
!git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

fatal: destination path 'DrugReviewAnalysis' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd

df1 = pd.read_csv('/content/DrugReviewAnalysis/drugsComTrain_raw.tsv', sep="\t")

df_train = pd.read_csv('/content/DrugReviewAnalysis/drugsTrain_processed.csv', sep="\t")

df_test = pd.read_csv('/content/DrugReviewAnalysis/drugsTest_processed.csv', sep="\t")

In [4]:
df_train

,drugName,condition,rating,date,usefulCount,timestamp,processed_review
0,valsartan,left ventricular dysfunction,9.0,"May 20, 2012",27,2012-05-20,side effect take combination bystolic 5 mg fi...
1,guanfacine,adhd,8.0,"April 27, 2010",192,2010-04-27,son halfway fourth week intuniv became concer...
2,lybrel,birth control,5.0,"December 14, 2009",17,2009-12-14,used take another oral contraceptive 21 pill ...
3,ortho evra,birth control,8.0,"November 3, 2015",10,2015-11-03,first time using form birth control m glad we...
4,buprenorphine / naloxone,opiate dependence,9.0,"November 27, 2016",37,2016-11-27,suboxone completely turned life around feel h...
...,...,...,...,...,...,...,...
159493,campral,alcohol dependence,10.0,"May 31, 2015",125,2015-05-31,wrote first report midoctober 2014 alcohol si...
159494,metoclopramide,nausea/vomiting,1.0,"November 1, 2011",34,2011-11-01,given iv surgey immediately became anxious co...
159495,orencia,rheumatoid arthritis,2.0,"March 15, 2014",35,2014-03-15,limited improvement 4 month developed bad ras...
159496,thyroid desiccated,underactive thyroid,10.0,"September 19, 2015",79,2015-09-19,ve thyroid medication 49 year spent first 38 ...


In [5]:
X_train = df_train.drop(columns=["rating"])
y_train = df_train["rating"]

X_test = df_test.drop(columns=["rating"])
y_test = df_test["rating"]

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'processed_review'),
        ('numeric', StandardScaler(), ['usefulCount']),
        ('categorical', OneHotEncoder(handle_unknown='ignore'), ['drugName','condition'])
    ],
    remainder='drop'
)

# Choose the models
pipeline = Pipeline([
    ('preprocessor', preprocessor),
#     ('classifier', LogisticRegression(max_iter=1000, n_jobs=-1, verbose=0)),
    # ('regressor', DecisionTreeRegressor()),
    # ('classifier', RandomForestClassifier(n_estimators=100)),
    # ('classifier', SVC()),
    # ('classifier', KNeighborsClassifier(n_neighbors=5))
])

print(pipeline)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text', TfidfVectorizer(),
                                                  'processed_review'),
                                                 ('numeric', StandardScaler(),
                                                  ['usefulCount']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['drugName',
                                                   'condition'])]))])


In [7]:
test = preprocessor.fit_transform(X_train)
test2 = preprocessor.fit_transform(X_test)

In [8]:
print(test[159495])

  (0, 12384)	0.18153304498871597
  (0, 17948)	0.2962848145921012
  (0, 20494)	0.3080099378246542
  (0, 31302)	0.30270425661257705
  (0, 34829)	0.42013148850311877
  (0, 36424)	0.40897183587096825
  (0, 36561)	0.1738649397418817
  (0, 38270)	0.13299008859300085
  (0, 47018)	0.3371645675867346
  (0, 47688)	0.4328941172044735
  (0, 62323)	0.1864474010611881
  (0, 64587)	1.0
  (0, 66393)	1.0


In [9]:
A = X_train['processed_review'][159495].split()
print(A)

['limited', 'improvement', '4', 'month', 'developed', 'bad', 'rash', 'md', 'refused', 'continue', 'medication']


In [10]:
len(A)

11

In [11]:
A=[]

for i in test:
  A.append(i.mean())

In [12]:
B=[]

for i in test2:
  B.append(i.mean())

In [13]:
X_train['review_score'] = A
X_test['review_score'] = B

In [14]:
X_train

,drugName,condition,date,usefulCount,timestamp,processed_review,review_score
0,valsartan,left ventricular dysfunction,"May 20, 2012",27,2012-05-20,side effect take combination bystolic 5 mg fi...,0.000068
1,guanfacine,adhd,"April 27, 2010",192,2010-04-27,son halfway fourth week intuniv became concer...,0.000199
2,lybrel,birth control,"December 14, 2009",17,2009-12-14,used take another oral contraceptive 21 pill ...,0.000119
3,ortho evra,birth control,"November 3, 2015",10,2015-11-03,first time using form birth control m glad we...,0.000099
4,buprenorphine / naloxone,opiate dependence,"November 27, 2016",37,2016-11-27,suboxone completely turned life around feel h...,0.000127
...,...,...,...,...,...,...,...
159493,campral,alcohol dependence,"May 31, 2015",125,2015-05-31,wrote first report midoctober 2014 alcohol si...,0.000175
159494,metoclopramide,nausea/vomiting,"November 1, 2011",34,2011-11-01,given iv surgey immediately became anxious co...,0.000102
159495,orencia,rheumatoid arthritis,"March 15, 2014",35,2014-03-15,limited improvement 4 month developed bad ras...,0.000078
159496,thyroid desiccated,underactive thyroid,"September 19, 2015",79,2015-09-19,ve thyroid medication 49 year spent first 38 ...,0.000156


In [15]:
X_test

,drugName,condition,date,usefulCount,timestamp,processed_review,review_score
0,mirtazapine,depression,"February 28, 2012",22,2012-02-28,ve tried antidepressant year citalopram fluox...,0.000171
1,mesalamine,"crohn's disease, maintenance","May 17, 2009",17,2009-05-17,son crohn s disease done well asacol complain...,0.000152
2,bactrim,urinary tract infection,"September 29, 2017",3,2017-09-29,quick reduction symptom,0.000071
3,contrave,weight loss,"March 5, 2017",35,2017-03-05,contrave combine drug used alcohol smoking op...,0.000223
4,cyclafem 1 / 35,birth control,"October 22, 2015",4,2015-10-22,birth control one cycle reading review type s...,0.000184
...,...,...,...,...,...,...,...
53195,tamoxifen,"breast cancer, prevention","September 13, 2014",43,2014-09-13,taken tamoxifen 5 year side effect severe swe...,0.000190
53196,escitalopram,anxiety,"October 8, 2016",11,2016-10-08,ve taking lexapro escitaploprgram since febru...,0.000189
53197,levonorgestrel,birth control,"November 15, 2010",7,2010-11-15,m married 34 year old kid taking pill hassle ...,0.000201
53198,tapentadol,pain,"November 28, 2011",20,2011-11-28,prescribed nucynta severe neckshoulder pain t...,0.000128


In [16]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)

encoded_data1 = encoder.fit_transform(X_train[['drugName','condition']])

encoded_columns = encoder.get_feature_names_out(['drugName', 'condition'])

df_encoded_train = pd.DataFrame(encoded_data1, columns=encoded_columns)

encoded_data2 = encoder.fit_transform(X_test[['drugName','condition']])

encoded_columns = encoder.get_feature_names_out(['drugName', 'condition'])

df_encoded_test = pd.DataFrame(encoded_data2, columns=encoded_columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
df_encoded_train

,drugName_a + d cracked skin relief,drugName_a / b otic,drugName_abacavir / dolutegravir / lamivudine,drugName_abacavir / lamivudine / zidovudine,drugName_abatacept,drugName_abilify,drugName_abilify discmelt,drugName_abilify maintena,drugName_abiraterone,drugName_abobotulinumtoxina,...,condition_wegener's granulomatosis,condition_weight loss,condition_wilson's disease,condition_wolff-parkinson-white syndrome,condition_women (minoxidil),condition_women (oxybutynin),condition_wound cleansing,condition_xerostomia,condition_zen shoulde,condition_zollinger-ellison syndrome
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_encoded_train[['review_score', 'usefulCount']] = X_train[['review_score', 'usefulCount']]
df_encoded_test[['review_score', 'usefulCount']] = X_test[['review_score', 'usefulCount']]

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, precision_score, recall_score
from keras.utils import to_categorical

In [ ]:
model = Sequential()
model.add(Dense(units = 16, activation = 'relu', input_dim = df_encoded_train.shape[1]))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(y_train.shape[0],activation='sigmoid'))

from sklearn.model_selection import train_test_split

In [ ]:
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy")

model.fit(df_encoded_train,y_train,epochs = 2, batch_size= 500, validation_split = 0.2)

yhat = model.predict(df_encoded_test)